# Коллаборативная фильтрация

1. **Используем датасет MovieLens**

In [6]:
import pandas as pd

# Загрузка данных оценок
ratings_df = pd.read_csv('C:/Users/Yaros/Downloads/ml-latest/ml-latest/ratings.csv')

In [7]:
from surprise import Dataset, Reader

# Определение формата данных
reader = Reader(rating_scale=(0.5, 5.0))

# Загрузка данных в Surprise
data = Dataset.load_from_df(ratings_df[['userId', 'movieId', 'rating']], reader)

2. **Выбор и обучение модели**

Для начала, можно использовать алгоритм SVD, который часто показывает хорошие результаты на задачах рекомендаций. Также проведем простой поиск по сетке для оптимизации гиперпараметров модели SVD, чтобы попытаться достичь RMSE на уровне 0,87 или ниже.

In [8]:
from surprise.model_selection import cross_validate, train_test_split
from surprise import SVD, accuracy
from surprise.model_selection import GridSearchCV

# Разделение данных на обучающую и тестовую выборки
trainset, testset = train_test_split(data, test_size=0.25)

# Обучение модели SVD
model = SVD()

# Поиск по сетке для настройки гиперпараметров SVD
param_grid = {
    'n_epochs': [5, 10, 20], 
    'lr_all': [0.002, 0.005],
    'reg_all': [0.4, 0.6]
}
gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=3)
gs.fit(data)

# Лучшие параметры
print(gs.best_params['rmse'])

# Обучение модели с лучшими параметрами на полном тренировочном наборе
best_model = gs.best_estimator['rmse']
trainset = data.build_full_trainset()
best_model.fit(trainset)

# Предсказания на тестовой выборке
predictions = best_model.test(testset)

# Вычисление RMSE
rmse = accuracy.rmse(predictions)

{'n_epochs': 20, 'lr_all': 0.002, 'reg_all': 0.4}
RMSE: 0.8756


In [9]:
print(rmse)

0.875633752533992


Модель SVD (Singular Value Decomposition) была выбрана из-за её популярности и эффективности в задачах коллаборативной фильтрации. Этот алгоритм часто демонстрирует высокую точность предсказаний в системах рекомендаций.  
  
Поиск по сетке для настройки гиперпараметров (GridSearchCV) использовался для оптимизации параметров модели SVD, таких как количество эпох обучения, скорость обучения и параметры регуляризации, с целью достижения наилучшего значения RMSE.